In [ ]:
import os
import pandas as pd
from PIL import Image

# Paths
path2train = "data/train"
excel_path = os.path.join(path2train, "image_labels.xlsx")

# Function to create labels DataFrame and save to Excel
def create_labels_dataframe(path2train, excel_path):
    data = []
    index = 0
    for img_name in os.listdir(path2train):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(path2train, img_name)
            img = Image.open(img_path)
            img = img.resize((224, 224))  # Resize image to (224, 224)
            img.save(img_path)  # Save resized image, overwriting the original
            
            label = 0 if 'cat' in img_name else 1  # Assign label: 0 for cats, 1 for dogs
            data.append((index, img_name, label))
            index += 1
    
    labels_df = pd.DataFrame(data, columns=['index', 'imgName', 'label'])
    labels_df.to_excel(excel_path, index=False)  # Save to Excel file
    return labels_df

# Create labels dataframe
labels_df = create_labels_dataframe(path2train, excel_path)


from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms.functional as TF


# Dataset class
class AMD_dataset(Dataset):
    def __init__(self, path2data, transform=None, img_size=(224, 224)):
        self.path2labels = os.path.join(path2data, "image_labels.xlsx")
        self.labels_df = pd.read_excel(self.path2labels)
        self.labels = self.labels_df["label"].values
        self.img_names = self.labels_df["imgName"]
        self.ids = self.labels_df.index
        self.transform = transform
        self.img_size = img_size
        self.path2train = path2data

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        img_path = os.path.join(self.path2train, img_name)
        image = Image.open(img_path).resize(self.img_size)  # Resize image
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def transformer(image):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image = TF.to_tensor(image)
    return image


# Create dataset instances
amd_ds = AMD_dataset(path2train, transformer)
print(len(amd_ds))

from sklearn.model_selection import ShuffleSplit
from torch.utils.data import Subset, DataLoader

# Split the dataset into training and validation sets
sss = ShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
indices = range(len(amd_ds))
for train_index, val_index in sss.split(indices):
    print(len(train_index))
    print("-" * 10)
    print(len(val_index))

# Create training and validation subsets
train_ds = Subset(amd_ds, train_index)
val_ds = Subset(amd_ds, val_index)

print(len(train_ds))
print(len(val_ds))

# DataLoader
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=3, shuffle=True)

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchsummary import summary

# CNN Model
class Net(nn.Module):
    def __init__(self, params):
        super(Net, self).__init__()
        C_in, H_in, W_in = params["input_shape"]
        init_f = params["initial_filters"]
        num_fc1 = params["num_fc1"]
        num_classes = params["num_classes"]
        self.dropout_rate = params["dropout_rate"]
        self.conv1 = nn.Conv2d(C_in, init_f, kernel_size=3)
        h, w = self.findConv2dOutShape(H_in, W_in, self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2 * init_f, kernel_size=3)
        h, w = self.findConv2dOutShape(h, w, self.conv2)
        self.conv3 = nn.Conv2d(2 * init_f, 4 * init_f, kernel_size=3)
        h, w = self.findConv2dOutShape(h, w, self.conv3)
        self.conv4 = nn.Conv2d(4 * init_f, 8 * init_f, kernel_size=3)
        h, w = self.findConv2dOutShape(h, w, self.conv4)
        # compute the flatten size
        self.num_flatten = h * w * 8 * init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def findConv2dOutShape(self, H_in, W_in, conv, pool=2):  # get conv arguments
        kernel_size = conv.kernel_size
        stride = conv.stride
        padding = conv.padding
        dilation = conv.dilation
        H_out = np.floor((H_in + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1)
        W_out = np.floor((W_in + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1)
        if pool:
            H_out /= pool
            W_out /= pool
        return int(H_out), int(W_out)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, self.num_flatten)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, self.dropout_rate, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Model parameters
params_model = {
    "input_shape": (3, 224, 224),
    "initial_filters": 8,
    "num_fc1": 100,
    "dropout_rate": 0.25,
    "num_classes": 2,
}

model = Net(params_model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(model)
print(next(model.parameters()).device)

summary(model, input_size=(3, 224, 224), device=device.type)


import torch.optim as optim

# Training and validation loops
def train_model(model, train_dl, val_dl, device, num_epochs=10, lr=0.001):
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        
        for images, labels in train_dl:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_train_loss = running_loss / len(train_dl)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")
        
        # Validate the model
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for images, labels in val_dl:
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item()
                
                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        avg_val_loss = val_running_loss / len(val_dl)
        val_accuracy = 100 * correct / total
        print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
    
    print("Training complete")

# Train the model
train_model(model, train_dl, val_dl, device)


